In [1]:
# Imports
import os
from django.apps import apps as django_apps

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
from flourish_child.models import Appointment

child_idx = 'B142-040990678-0-10'
upper_visit = 2008
visit = 2005

while visit <= upper_visit:
    try:
        old_visit = ChildVisit.objects.get(subject_identifier=child_idx, visit_code=visit)
    except ChildVisit.DoesNotExist:
        print(visit)
        break
    else:
        appt = old_visit.appointment
        new_visit_code = visit - 1
        new_appt = Appointment.objects.filter(
            subject_identifier=child_idx, visit_code=new_visit_code)
        new_appt.update(
            appt_datetime=appt.appt_datetime,
            appt_type=appt.appt_type,
            appt_status=appt.appt_status,
            appt_reason=appt.appt_reason)
        old_visit.appointment = new_appt[0]
        old_visit.save()
        visit += 1

In [2]:
caregiver_idx = 'B142-040990678-0'
visit_codes = ['2004M', '2005M']
visits = MaternalVisit.objects.filter(
    subject_identifier=caregiver_idx, visit_code__in=visit_codes).values_list('id', flat=True)

metadata2 = CrfMetadata.objects.filter(
    subject_identifier=caregiver_idx,
    visit_code=visit_codes[1],
    entry_status='KEYED').values_list('model', flat=True)

exclude_fields = ['_state', 'created', 'modified', 'user_created', 'user_modified',
                  'report_datetime', 'maternal_visit_id', 'id', 'hostname_created',
                  'hostname_modified', 'revision', 'device_created', 'device_modified',
                  'site_id', 'consent_version']

print(metadata2)
for model in metadata2:
    model_cls = django_apps.get_model(model)
    try:
        obj1, obj2 = model_cls.objects.filter(maternal_visit__in=visits)
    except ValueError:
        obj1 = model_cls.objects.filter(maternal_visit__in=visits)[0]
        obj2 = None

    if model == 'flourish_caregiver.hivrapidtestcounseling':
        obj1.delete()
        obj2.delete()

    for key, value in obj1.__dict__.items():
        if key in exclude_fields:
            continue
        obj2_value = getattr(obj2, key, None)
        if obj2 and obj2_value != value:
            print(model, key, value, obj2_value)
            setattr(obj1, key, obj2_value)

        if not obj2:
            obj1.maternal_visit = MaternalVisit.objects.get(id=visits[0])

    try:
        obj1.save_base(raw=True)
    except Exception as e:
        print(model, e)
    else:
        if obj2:
            obj2.delete()
            


<QuerySet ['flourish_caregiver.sociodemographicdata', 'flourish_caregiver.medicalhistory', 'flourish_caregiver.covid19', 'flourish_caregiver.interviewfocusgroupinterest']>


In [10]:
from django.db.models.signals import post_save, post_delete
post_delete.disconnect(dispatch_uid='metadata_reset_on_post_delete')

visit = MaternalVisit.objects.filter(subject_identifier='B142-040990678-0', visit_code='2005M')
HIVRapidTestCounseling.objects.filter(maternal_visit=visit[0]).delete()
visit.delete()

(1, {'flourish_caregiver.MaternalVisit': 1})

In [11]:
from edc_appointment.models import Appointment

maternal_idx = 'B142-040990678-0'
upper_visit = 2008
visit = 2006

while visit <= upper_visit:
    try:
        old_visit = MaternalVisit.objects.get(subject_identifier=maternal_idx, visit_code=f'{visit}M')
    except MaternalVisit.DoesNotExist:
        print(visit)
        break
    else:
        appt = old_visit.appointment
        new_visit_code = visit - 1
        new_appt = Appointment.objects.filter(
            subject_identifier=maternal_idx, visit_code=f'{new_visit_code}M')
        new_appt.update(
            appt_datetime=appt.appt_datetime,
            appt_type=appt.appt_type,
            appt_status=appt.appt_status,
            appt_reason=appt.appt_reason)
        old_visit.appointment = new_appt[0]
        old_visit.save()
        visit += 1

In [12]:
Appointment.objects.filter(subject_identifier=maternal_idx, visit_code=f'2008M').update(appt_status='in_progress')

1